In [722]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline


In [723]:
input_file = "winequality-red.csv"
data = pd.read_csv(input_file)
tmp = data

#### 데이터 부족으로 판단하여 2배 증가 X (불가능)

In [724]:
# data = pd.concat([data,data], ignore_index=True)

In [725]:
data = data.drop("residual sugar", axis=1)
data = data.drop("density", axis=1)

In [726]:
corr_matrix = data.corr()
corr_matrix['quality']

fixed acidity           0.124052
volatile acidity       -0.390558
citric acid             0.226373
chlorides              -0.128907
free sulfur dioxide    -0.050656
total sulfur dioxide   -0.185100
pH                     -0.057731
sulphates               0.251397
alcohol                 0.476166
quality                 1.000000
Name: quality, dtype: float64

In [727]:
# from itertools import combinations

# # features = housing_train[housing_train.drop('price', axis=1).columns]
# # housing_train = housing_train[['price','sqft_living//waterfront','(waterfront//(waterfront/sqft_living))','(sqft_living15*((sqft_living**2)//(waterfront//(waterfront/sqft_living))))','(waterfront//waterfront/sqft_living)','(sqft_living15*(waterfront//(waterfront/sqft_living)))']]
# features = data.drop('quality', axis=1)

# combinations_list = list(combinations(features.columns, 2))
# # # 조합된 특성들의 평균을 나타내는 새로운 특성들 생성
# for combo in combinations_list:
#     print("1Number of columns in housing_train:", len(data.columns))
#     print(f"0Number of columns in housing_train:", len(data.columns), f"{combo}")
#     data.replace([np.inf, -np.inf], np.nan, inplace=True)

#     feature_name3 = f"({combo[0]}*{combo[1]})"
#     data[feature_name3] = (features[combo[0]] * features[combo[1]])

#     feature_name3 = f"({combo[0]}**{combo[1]})"
#     data[feature_name3] = (features[combo[0]] ** features[combo[1]])

#     feature_name3 = f"({combo[0]}//{combo[1]})"
#     data[feature_name3] = (features[combo[0]] // features[combo[1]])

#     feature_name3 = f"({combo[0]}+2{combo[1]})"
#     data[feature_name3] = (features[combo[0]] + 2*features[combo[1]])

#     feature_name3 = f"(2{combo[0]}+{combo[1]})"
#     data[feature_name3] = (2*features[combo[0]] + features[combo[1]])

#     feature_name3 = f"(({combo[0]}+{combo[1]})/2)"
#     data[feature_name3] = (features[combo[0]] + features[combo[1]])/2

# # features = data.drop('quality', axis=1)

# # combinations_list = list(combinations(features.columns, 2))
# # # # 조합된 특성들의 평균을 나타내는 새로운 특성들 생성
# # for combo in combinations_list:
# #     print("1Number of columns in housing_train:", len(data.columns))
# #     print(f"0Number of columns in housing_train:", len(data.columns), f"{combo}")
# #     data.replace([np.inf, -np.inf], np.nan, inplace=True)

# #     feature_name3 = f"({combo[0]}*{combo[1]})"
# #     data[feature_name3] = (features[combo[0]] * features[combo[1]])

# #     feature_name3 = f"({combo[0]}**{combo[1]})"
# #     data[feature_name3] = (features[combo[0]] ** features[combo[1]])

# #     feature_name3 = f"({combo[0]}//{combo[1]})"
# #     data[feature_name3] = (features[combo[0]] // features[combo[1]])

# #     feature_name3 = f"({combo[0]}+2{combo[1]})"
# #     data[feature_name3] = (features[combo[0]] + 2*features[combo[1]])

# #     feature_name3 = f"(2{combo[0]}+{combo[1]})"
# #     data[feature_name3] = (2*features[combo[0]] + features[combo[1]])

# #     feature_name3 = f"(({combo[0]}+{combo[1]})/2)"
# #     data[feature_name3] = (features[combo[0]] + features[combo[1]])/2


In [728]:
# corr = data.corr()
# plt.figure(figsize=(20, 9))
# k = 12 #number of variables for heatmap
# cols = corr.nlargest(k, 'quality')['quality'].index
# cm = np.corrcoef(data[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values,cmap="Blues")
# plt.show()

In [729]:
# data.isnull().sum()

In [730]:
# data.describe()

In [731]:
# additional calculations
data['total_acidity'] = data['fixed acidity'] + data['volatile acidity']
data['average_acidity'] = (data['fixed acidity'] + data['volatile acidity']) / 2
data['total_minerals'] = data['chlorides'] + data['sulphates']
data['average_minerals'] = (data['chlorides'] + data['sulphates']) / 2
data['non_free_sulfur_dioxide'] = np.abs(data['total sulfur dioxide'] - data['free sulfur dioxide'])

# calculate percentage_free_sulfur without using CombineWithReferenceFeature
data['percentage_free_sulfur'] = data['free sulfur dioxide'] / data['total sulfur dioxide']
data['percentage_free_sulfur'].replace([np.inf, -np.inf], np.nan, inplace=True)
data['percentage_free_sulfur'].fillna(0, inplace=True)  # handling possible NaN values

# additional calculations
data['percentage_salt_sulfur'] = data['sulphates'] / data['free sulfur dioxide']
data['percentage_salt_sulfur'].replace([np.inf, -np.inf], np.nan, inplace=True)
data['percentage_salt_sulfur'].fillna(0, inplace=True)  # handling possible NaN values

# data['alcohol+sulphates'] = data['alcohol'] + data['sulphates']

data["balance"] = data["alcohol"] - data["sulphates"] - data["volatile acidity"]
data['total_minerals'] = data['chlorides'] + data['sulphates']
data['sulph_sub_chlor']= data['sulphates']- data['chlorides']

# data['(2sulphates+alcohol)'] = 2*data['sulphates'] + data['alcohol']
data['(2citric acid+alcohol)'] = 2*data['citric acid'] + data['alcohol']
# data['((sulphates+alcohol)/2)'] = ((data['sulphates']+data['alcohol'])/2)
# (2sulphates+alcohol)                           0.516197
# (2citric acid+alcohol)                         0.507274 X
# ((sulphates+alcohol)/2)                        0.502502 X


data['1'] = data['balance'] + 2*data['sulph_sub_chlor']
# (balance+2sulph_sub_chlor)                          0.540696
# (2sulphates+balance)                                0.538952 X
# (2sulph_sub_chlor+(2citric acid+alcohol))           0.538115 X 

data['2'] = data['volatile acidity']**data['sulph_sub_chlor']
# (volatile acidity**sulph_sub_chlor)             -0.446816

data['3'] = (data['alcohol']/data['1'])
# (alcohol/1)                                  -0.404521


In [732]:
corr_matrix = data.corr()
corr_matrix['quality'].sort_values(ascending=False)

quality                    1.000000
1                          0.540696
(2citric acid+alcohol)     0.507274
balance                    0.483355
alcohol                    0.476166
sulph_sub_chlor            0.307735
sulphates                  0.251397
citric acid                0.226373
percentage_free_sulfur     0.194113
average_minerals           0.190327
total_minerals             0.190327
percentage_salt_sulfur     0.128885
fixed acidity              0.124052
average_acidity            0.085709
total_acidity              0.085709
free sulfur dioxide       -0.050656
pH                        -0.057731
chlorides                 -0.128907
total sulfur dioxide      -0.185100
non_free_sulfur_dioxide   -0.205463
volatile acidity          -0.390558
2                         -0.446816
3                         -0.448255
Name: quality, dtype: float64

In [733]:
# from itertools import combinations

# # features = housing_train[housing_train.drop('price', axis=1).columns]
# # housing_train = housing_train[['price','sqft_living//waterfront','(waterfront//(waterfront/sqft_living))','(sqft_living15*((sqft_living**2)//(waterfront//(waterfront/sqft_living))))','(waterfront//waterfront/sqft_living)','(sqft_living15*(waterfront//(waterfront/sqft_living)))']]
# features = data.drop('quality', axis=1)

# combinations_list = list(combinations(features.columns, 2))
# # # 조합된 특성들의 평균을 나타내는 새로운 특성들 생성
# for combo in combinations_list:
#     print("1Number of columns in housing_train:", len(data.columns))
#     print(f"0Number of columns in housing_train:", len(data.columns), f"{combo}")
#     data.replace([np.inf, -np.inf], np.nan, inplace=True)

#     feature_name3 = f"({combo[0]}*{combo[1]})"
#     data[feature_name3] = (features[combo[0]] * features[combo[1]])

#     feature_name3 = f"({combo[0]}**{combo[1]})"
#     data[feature_name3] = (features[combo[0]] ** features[combo[1]])

#     feature_name3 = f"({combo[0]}/{combo[1]})"
#     data[feature_name3] = (features[combo[0]] // features[combo[1]])

#     feature_name3 = f"({combo[0]}+2{combo[1]})"
#     data[feature_name3] = (features[combo[0]] + 2*features[combo[1]])

#     feature_name3 = f"(2{combo[0]}+{combo[1]})"
#     data[feature_name3] = (2*features[combo[0]] + features[combo[1]])

#     feature_name3 = f"(({combo[0]}+{combo[1]})/2)"
#     data[feature_name3] = (features[combo[0]] + features[combo[1]])/2

In [734]:
corr_matrix = data.corr()
corr_matrix['quality'].sort_values(ascending=False)

quality                    1.000000
1                          0.540696
(2citric acid+alcohol)     0.507274
balance                    0.483355
alcohol                    0.476166
sulph_sub_chlor            0.307735
sulphates                  0.251397
citric acid                0.226373
percentage_free_sulfur     0.194113
average_minerals           0.190327
total_minerals             0.190327
percentage_salt_sulfur     0.128885
fixed acidity              0.124052
average_acidity            0.085709
total_acidity              0.085709
free sulfur dioxide       -0.050656
pH                        -0.057731
chlorides                 -0.128907
total sulfur dioxide      -0.185100
non_free_sulfur_dioxide   -0.205463
volatile acidity          -0.390558
2                         -0.446816
3                         -0.448255
Name: quality, dtype: float64

In [735]:
corr_matrix['quality'].sort_values(ascending=True).head(30)

3                         -0.448255
2                         -0.446816
volatile acidity          -0.390558
non_free_sulfur_dioxide   -0.205463
total sulfur dioxide      -0.185100
chlorides                 -0.128907
pH                        -0.057731
free sulfur dioxide       -0.050656
total_acidity              0.085709
average_acidity            0.085709
fixed acidity              0.124052
percentage_salt_sulfur     0.128885
total_minerals             0.190327
average_minerals           0.190327
percentage_free_sulfur     0.194113
citric acid                0.226373
sulphates                  0.251397
sulph_sub_chlor            0.307735
alcohol                    0.476166
balance                    0.483355
(2citric acid+alcohol)     0.507274
1                          0.540696
quality                    1.000000
Name: quality, dtype: float64

In [736]:
train_data = data.drop('quality', axis=1)
train_data = train_data.dropna(axis=1)
train_target = data['quality']

In [737]:
train_data = pd.DataFrame(train_data)

In [738]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_data)

train_data = scaler.transform(train_data)

In [739]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesRegressor

# extra_trees_clf = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='squared_error',
#                     max_depth=30, max_features=1.0, max_leaf_nodes=None,
#                     max_samples=None, min_impurity_decrease=0.0,
#                     min_samples_leaf=1, min_samples_split=2,
#                     min_weight_fraction_leaf=0.0, n_estimators=605, n_jobs=-1,
#                     oob_score=False, random_state=42, verbose=0,
#                     warm_start=False)


# # lr = LogisticRegression()
# for i,e in enumerate(train_data.columns):
#     extra_trees_clf.fit(train_data[e].values[:,np.newaxis], train_target.values)
#     plt.title("Best fit line")
#     plt.xlabel(str(e))
#     plt.ylabel('quality')
#     plt.scatter(train_data[e].values[:,np.newaxis], train_target)
#     plt.plot(train_data[e].values[:,np.newaxis], extra_trees_clf.predict(train_data[e].values[:,np.newaxis]),color='r')
#     plt.show()

In [740]:
# seed = 42
# np.random.seed(seed)

# sh_in = np.random.permutation(len(train_data))
# train_data = train_data.values[sh_in]
# train_target = train_target.values[sh_in]

## 이미 방향성을 지니고 있기 때문에 PCA는 적용하지 않는다

In [741]:
# from sklearn.ensemble import RandomForestRegressor

# # RandomForest 모델 생성 및 학습
# rf_regressor = RandomForestRegressor(random_state=42)
# rf_regressor.fit(train_data, train_target)

In [742]:
# from sklearn.ensemble import AdaBoostRegressor

# # Ada Boosting 모델 생성 및 학습
# ada_regressor = AdaBoostRegressor(random_state=42)
# ada_regressor.fit(train_data, train_target)

In [743]:
# k-폴드 CV의 평균을 계산하기 위한 함수
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [744]:
# from sklearn.preprocessing import PolynomialFeatures
# poly_features = PolynomialFeatures(degree=2, include_bias=False)
# train_data = poly_features.fit_transform(train_data)


In [745]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_data, train_target)
lin_reg.intercept_, lin_reg.coef_

(5.608943076739518,
 array([-1.36046815e+12,  8.96767726e+11,  2.21653230e+12,  2.47192708e+12,
         3.22967662e+11, -1.01567558e+12, -6.60404429e-02,  3.52158536e+13,
        -1.06964220e+11,  1.75471548e+11,  1.15604673e+12, -1.26455066e+13,
        -1.17876607e+13,  8.35389286e+11,  4.96678482e-02,  5.91799789e-03,
         2.58672564e+12, -1.29122215e+13, -6.68016449e+12,  4.03507061e+12,
        -1.39097342e-02,  3.75778769e-01]))

In [746]:
# plot_learning_curves(lin_reg, train_data, train_target)

In [747]:
train_data

array([[-0.52835961,  0.96187667, -1.39147228, ..., -1.0869892 ,
         0.99087692,  0.97277383],
       [-0.29854743,  1.96744245, -1.39147228, ..., -0.83371914,
         1.53112407,  1.34439663],
       [-0.29854743,  1.29706527, -1.18607043, ..., -0.74644365,
         1.09391486,  0.93091944],
       ...,
       [-1.1603431 , -0.09955388, -0.72391627, ...,  0.60718214,
        -0.29211499, -0.50618085],
       [-1.39015528,  0.65462046, -0.77526673, ..., -0.22535761,
         0.46531067,  0.09802417],
       [-1.33270223, -1.21684919,  1.02199944, ...,  0.71670433,
        -1.13725942, -0.92482707]])

In [748]:
# extra_trees_clf = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='squared_error',
#                     max_depth=None, max_features=1.0, max_leaf_nodes=None,
#                     max_samples=None, min_impurity_decrease=0.0,
#                     min_samples_leaf=1, min_samples_split=2,
#                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
#                     oob_score=False, random_state=123, verbose=0,
#                     warm_start=False)
# extra_trees_clf.fit(train_data, train_target)
# extra_trees_clf.feature_importances_

In [749]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.ensemble import StackingRegressor
# from xgboost import XGBRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import ExtraTreesRegressor

# from sklearn.model_selection import cross_val_score

# # 기본 모델 정의
# base_models = [
#     # ('linear', LinearRegression()),
#     ('rf',RandomForestRegressor(n_estimators=10, random_state=42)),
#     # ('tree', DecisionTreeRegressor()),
#     # ('et', ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='squared_error',
#     #                 max_depth=30, max_features=1.0, max_leaf_nodes=None,
#     #                 max_samples=None, min_impurity_decrease=0.0,
#     #                 min_samples_leaf=1, min_samples_split=2,
#     #                 min_weight_fraction_leaf=0.0, n_estimators=605, n_jobs=-1,
#     #                 oob_score=False, random_state=42, verbose=0,
#     #                 warm_start=False)),
#     ('xgb',XGBRegressor()),
#     ('hist',HistGradientBoostingRegressor())
#     # ('knn', KNeighborsRegressor()),
#     # ('svr', SVR()),
#     # ('gradient_boosting', GradientBoostingRegressor())
# ]

# # 메타 모델 정의
# # meta_model = RandomForestRegressor(n_estimators=10, random_state=42)
# meta_model = SVR()
# # 스태킹 모델 정의
# stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# # 각 모델 훈련 및 예측
# for name, model in base_models + [('stacking', stacking_model)]:
#     model.fit(train_data, train_target)
#     # y_pred = model.predict(train_data)
#     # mse = mean_squared_error(y_val, y_pred)
#     # print(f"{name} MSE: {mse}")

# # 최종 스태킹 모델 훈련
# stacking_model.fit(train_data, train_target)

# scores = cross_val_score(stacking_model, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

In [750]:
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.model_selection import cross_val_score

# extra_trees_clf = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='squared_error',
#                     max_depth=30, max_features=1.0, max_leaf_nodes=None,
#                     max_samples=None, min_impurity_decrease=0.0,
#                     min_samples_leaf=1, min_samples_split=2,
#                     min_weight_fraction_leaf=0.0, n_estimators=605, n_jobs=-1,
#                     oob_score=False, random_state=42, verbose=0,
#                     warm_start=False)

# scores = cross_val_score(extra_trees_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

Scores: [0.63246459 0.59018243 0.69025347 0.67308891 0.56489891 0.73273246
 0.66521203 0.64682767 0.60001301 0.69950872]
Mean: 0.6495182190019155
Standard deviation: 0.05031390577785743

In [751]:
# extra_trees_clf = ExtraTreesRegressor(max_depth = 30, max_features = 4, min_samples_leaf = 1, min_samples_split = 2,
#                                       n_estimators =2000, random_state=42)
# scores = cross_val_score(extra_trees_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

Scores: [0.63397486 0.59288826 0.66513791 0.66530967 0.55530522 0.7261735
 0.64796038 0.6461098  0.59657687 0.68522293]
Mean: 0.6414659391283845
Standard deviation: 0.04705027069060762

0.5525433647824474

In [752]:
# from sklearn.svm import LinearSVR
# from sklearn.model_selection import cross_val_score

# svr_clf = LinearSVR()

# scores = cross_val_score(svr_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

Scores: [0.68280167 0.64441914 0.67532206 0.66701238 0.61949657 0.72280197
 0.64714322 0.65848849 0.6057777  0.70139901]
Mean: 0.6624662204640509
Standard deviation: 0.03375193215292174

0.6331656216936654

In [753]:
# from sklearn.tree import DecisionTreeRegressor
# tree_reg = DecisionTreeRegressor(max_depth=3)
# tree_reg.fit(train_data, train_target)

# scores = cross_val_score(tree_reg, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

In [754]:
# from sklearn.ensemble import VotingRegressor

# voting_clf = VotingRegressor(estimators=[('Linear Regression', LinearRegression(n_jobs=-1)),
#                             ('Random Forest Regressor',
#                              RandomForestRegressor(n_jobs=-1,
#                                                    random_state=123))],
#                 n_jobs=-1)
# voting_clf.fit(train_data, train_target)

# scores = cross_val_score(voting_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

0.6304569173923081

In [755]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor


# voting_clf = VotingRegressor(estimators=[('Linear Regression', LinearRegression(n_jobs=-1)),
#                             ('Random Forest Regressor',
#                              RandomForestRegressor(n_jobs=-1,
#                                                    random_state=123)),
#                             ('Extra Trees Regressor',
#                              ExtraTreesRegressor(n_jobs=-1, random_state=123)),
#                             ('Extreme Gradient Boosting',
#                              XGBRegressor(base_score=None, booster='gbtree',
#                                           callbacks=None,
#                                           colsample_bylevel=None,
#                                           max_delta_step=None, max_depth=None,
#                                           max_leaves=None,
#                                           min_child_weight=None,
#                                           monotone_constraints=None,
#                                           multi_strategy=None,
#                                           n_estimators=None, n_jobs=-1,
#                                           num_parallel_tree=None,
#                                           random_state=123)),
#                             ('Gradient Boosting Regressor',
#                              GradientBoostingRegressor(random_state=123)),
#                             # ('Ridge Regression', Ridge(random_state=123)),
#                             # ('Bayesian Ridge', BayesianRidge())],
#                 ],n_jobs=-1)
# scores = cross_val_score(voting_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

0.6249271589107086

In [756]:
# voting_clf = VotingRegressor(estimators=[('Linear Regression', LinearRegression(n_jobs=-1)),
#                             ('Random Forest Regressor',
#                              RandomForestRegressor(n_jobs=-1,
#                                                    random_state=123)),
#                             # ('Extra Trees Regressor',
#                             #  ExtraTreesRegressor(n_jobs=-1, random_state=123)),
#                             ('Extreme Gradient Boosting',
#                              XGBRegressor(base_score=None, booster='gbtree',
#                                           callbacks=None,
#                                           colsample_bylevel=None,
#                                           max_leaves=None,
#                                           min_child_weight=None, 
#                                           monotone_constraints=None,
#                                           multi_strategy=None,
#                                           n_estimators=None, n_jobs=-1,
#                                           num_parallel_tree=None,
#                                           random_state=123)),
#                             ('Gradient Boosting Regressor',
#                              GradientBoostingRegressor(random_state=123)),
#                             # ('Light Gradient Boosting Machine',
#                             #  LGBMRegressor(random_state=123)),
#                             ],
#                 n_jobs=-1)

# scores = cross_val_score(voting_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

0.6277047020206654

In [757]:
# voting_clf = VotingRegressor(estimators=[('Linear Regression', LinearRegression(n_jobs=-1)),
#                             # ('Random Forest Regressor',
#                             #  RandomForestRegressor(n_jobs=-1,
#                             #                        random_state=123)),
#                             # ('Extra Trees Regressor',
#                             #  ExtraTreesRegressor(n_jobs=-1, random_state=123)),
#                             # ('Extreme Gradient Boosting',
#                             #  XGBRegressor(base_score=None, booster='gbtree',
#                             #               callbacks=None,
#                             #               colsample_bylevel=None,
#                             #               max_leaves=None,
#                             #               min_child_weight=None, 
#                             #               monotone_constraints=None,
#                             #               multi_strategy=None,
#                             #               n_estimators=None, n_jobs=-1,
#                             #               num_parallel_tree=None,
#                             #               random_state=123)),
#                             ('Gradient Boosting Regressor',
#                              GradientBoostingRegressor(random_state=123)),
#                             # ('Light Gradient Boosting Machine',
#                             #  LGBMRegressor(random_state=123)),
#                             ],
#                 n_jobs=-1)

# scores = cross_val_score(voting_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

0.6224548184464493

In [758]:
# from sklearn.svm import SVR
# voting_clf = VotingRegressor(estimators=[('Linear Regression', LinearRegression(n_jobs=-1)),
#                                          ('svr', SVR()),
#                             ('Gradient Boosting Regressor',
#                              GradientBoostingRegressor(random_state=123)),
#                             ],
#                 n_jobs=-1)

# scores = cross_val_score(voting_clf, train_data, train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

0.6182150233167303

#### 너무 오래걸림

In [ ]:
# import numpy as np
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.model_selection import GridSearchCV, cross_val_score

# seed = 553419
# np.random.seed(seed)

# # Assuming train_data and train_target are your input data and target variable
# shuffled_indices = np.random.permutation(len(train_data))
# shuffled_data = train_data[shuffled_indices]
# shuffled_target = train_target.values[shuffled_indices]

# # Hyperparameter grid for ExtraTreesRegressor
# param_grid = {
#     'n_estimators': range(1000, 3000, 100),
#     'max_depth': [29,30,31,32],
#     'max_features': [3,4,5],
#     'min_samples_leaf': [1,2],
#     'min_samples_split': [2],
#     'random_state': [42],
# }
# # Initialize ExtraTreesRegressor
# extra_trees_clf = ExtraTreesRegressor()

# # GridSearchCV to find the best hyperparameters
# grid_search = GridSearchCV(estimator=extra_trees_clf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
# grid_search.fit(shuffled_data, shuffled_target)

# # Print the best hyperparameters
# print("Best Hyperparameters:", grid_search.best_params_)

# # Extract the best model with the optimal hyperparameters
# best_extra_trees_clf = grid_search.best_estimator_

# # Cross-validation with the best model
# scores = cross_val_score(best_extra_trees_clf, shuffled_data, shuffled_target, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
# rmse_scores = np.sqrt(-scores)
# display_scores(rmse_scores)

In [ ]:
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.model_selection import cross_val_score

# seed = 553419
# np.random.seed(seed)
# shuffled_in = np.random.permutation(len(train_data))
# shuffled_train_data = train_data[shuffled_in]
# shuffled_train_target = train_target[shuffled_in]
# for i in range(1000, 10000, 100):
#     extra_trees_clf = ExtraTreesRegressor(max_depth = 30, max_features = 3, min_samples_leaf = 1, min_samples_split = 2,
#                                         n_estimators = i, random_state=42)

#     scores = cross_val_score(extra_trees_clf, sh_t_data, sh_t_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
#     rmse_scores = np.sqrt(-scores)
#     print(f"현재값 {i}")
#     display_scores(rmse_scores)

In [772]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score

seed = 553419
np.random.seed(seed)
shuffled_in = np.random.permutation(len(train_data))
shuffled_train_data = train_data[shuffled_in]
shuffled_train_target = train_target[shuffled_in]

extra_trees_clf = ExtraTreesRegressor(max_depth = 30, max_features = 3, min_samples_leaf = 1, min_samples_split = 2,
                                      n_estimators =1100, random_state=42)

scores = cross_val_score(extra_trees_clf, shuffled_train_data, shuffled_train_target, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [0.53273315 0.59300028 0.51516492 0.52162257 0.56635211 0.6256167
 0.46653252 0.5009357  0.53848157 0.4908538 ]
Mean: 0.535129330882258
Standard deviation: 0.045765609946900425


### 최종 0.5351887017868129